In [1]:
import sympy
from sympy.physics import vector
from IPython.display import display

sympy.init_printing(use_latex="mathjax")

In [16]:
N = vector.ReferenceFrame("N")
A, m, k, t, t_0, sigma_0, dv = sympy.symbols("A m k t t_0 \\sigma_0, {\\Delta}v", real=True, positive=True)
M1_x, M1_y, M2, v_x, v_y, vx_0, vy_0 = sympy.symbols("M_{1_x} M_{1_y} M_2 v_x v_y v_{x_0} v_{y_0}", real=True)
v = v_x*N.x + v_y*N.y
v_0 = vx_0*N.x + vy_0*N.y
M1 = M1_x*N.x + M1_y*N.y
sigma = A*sympy.exp(-m*vector.dot(v - v_0, v - v_0)/(2*k*(t - t_0)))

def eval_at_zero(expression):
    return expression.subs(vx_0, 0.0).subs(vy_0, 0.0).subs(t, 0.0)

def taylor(expression, variables = [t, vx_0, vy_0]):
    expansion = sympy.simplify(eval_at_zero(expression)/eval_at_zero(expression))
    for var in variables:
        expansion = expansion + sympy.simplify(eval_at_zero(sympy.diff(expression, var))*var/eval_at_zero(expression))
        
#    for var1 in variables:
#        for var2 in variables:
#            expansion = expansion + sympy.simplify(eval_at_zero(sympy.diff(sympy.diff(expression, var1), var2))*var1*var2/(2*eval_at_zero(expression)))
    return expansion*eval_at_zero(expression)

def zeroeth_moments(expression, vrange=[-dv, 0.0, dv]):
    moment = m -m
    for vx in vrange:
        for vy in vrange:
            moment = moment + (expression).subs(v_x, vx).subs(v_y, vy)
    return sympy.simplify(moment)

    
def first_moments(expression, vrange=[-dv, 0.0, dv]):
    moment = v - v
    for vx in vrange:
        for vy in vrange:
            moment = moment + (v*expression).subs(v_x, vx).subs(v_y, vy)
    return moment.simplify()

    
def second_moments(expression, vrange=[-dv, 0.0, dv]):
    moment = m - m
    for vx in vrange:
        for vy in vrange:
            moment = moment + (vector.dot(v, v)*expression).subs(v_x, vx).subs(v_y, vy)
    return sympy.simplify(moment)

def simp(expression):
    return sympy.simplify(sympy.powsimp(sympy.collect(sympy.collect(expression, M2), dv**2)))

In [3]:
linearization = taylor(sigma)
linearization

                                                           ⎛  2      2⎞
                                                         m⋅⎝vₓ  + v_y ⎠
  ⎛        ⎛  2      2⎞                               ⎞  ──────────────
  ⎜    m⋅t⋅⎝vₓ  + v_y ⎠   m⋅vₓ⋅v_{x_0}   m⋅v_y⋅v_{y_0}⎟      2⋅k⋅t₀    
A⋅⎜1 + ──────────────── - ──────────── - ─────────────⎟⋅ℯ              
  ⎜              2            k⋅t₀            k⋅t₀    ⎟                
  ⎝        2⋅k⋅t₀                                     ⎠                

In [4]:
zmoment = zeroeth_moments(linearization) - 1
zmoment

                                                                           2  
                2                   2                           m⋅{\Delta}v   
     m⋅{\Delta}v         m⋅{\Delta}v                            ────────────  
     ────────────        ────────────                        2     2⋅k⋅t₀     
        2⋅k⋅t₀               k⋅t₀           2⋅A⋅m⋅t⋅{\Delta}v ⋅ℯ              
4⋅A⋅ℯ             + 4⋅A⋅ℯ             + A + ──────────────────────────────── +
                                                             2                
                                                         k⋅t₀                 

                                2    
                     m⋅{\Delta}v     
                     ────────────    
                  2      k⋅t₀        
 4⋅A⋅m⋅t⋅{\Delta}v ⋅ℯ                
 ──────────────────────────────── - 1
                  2                  
              k⋅t₀                   

In [5]:
fmoment = first_moments(linearization) - M1
fmoment

⎛                           ⎛            2                 2⎞          ⎞
⎜                           ⎜ m⋅{\Delta}v       m⋅{\Delta}v ⎟          ⎟
⎜                           ⎜ ────────────      ────────────⎟          ⎟
⎜                         2 ⎜    2⋅k⋅t₀             k⋅t₀    ⎟          ⎟
⎜  2⋅A⋅m⋅v_{x_0}⋅{\Delta}v ⋅⎝ℯ             + 2⋅ℯ            ⎠          ⎟
⎜- ────────────────────────────────────────────────────────── - M_{1_x}⎟
⎝                             k⋅t₀                                     ⎠ n_x + ⎛                           ⎛            2                 2⎞          ⎞
⎜                           ⎜ m⋅{\Delta}v       m⋅{\Delta}v ⎟          ⎟
⎜                           ⎜ ────────────      ────────────⎟          ⎟
⎜                         2 ⎜    2⋅k⋅t₀             k⋅t₀    ⎟          ⎟
⎜  2⋅A⋅m⋅v_{y_0}⋅{\Delta}v ⋅⎝ℯ             + 2⋅ℯ            ⎠          ⎟
⎜- ────────────────────────────────────────────────────────── - M_{1_y}⎟
⎝                             k⋅t₀           

In [6]:
smoment = second_moments(linearization) - M2
smoment

               ⎛                    2                       2                 
               ⎜         m⋅{\Delta}v             m⋅{\Delta}v                  
               ⎜         ────────────            ────────────                 
             2 ⎜      2     2⋅k⋅t₀            2      k⋅t₀                    2
2⋅A⋅{\Delta}v ⋅⎝2⋅k⋅t₀ ⋅ℯ             + 4⋅k⋅t₀ ⋅ℯ             + m⋅t⋅{\Delta}v 
──────────────────────────────────────────────────────────────────────────────
                                                                2             
                                                            k⋅t₀              

             2                                2⎞     
  m⋅{\Delta}v                      m⋅{\Delta}v ⎟     
  ────────────                     ────────────⎟     
     2⋅k⋅t₀                     2      k⋅t₀    ⎟     
⋅ℯ             + 4⋅m⋅t⋅{\Delta}v ⋅ℯ            ⎠     
──────────────────────────────────────────────── - M₂
                                           

In [7]:
System_solution = sympy.solve([zmoment, fmoment.dot(N.x), fmoment.dot(N.y), smoment], (A, v_0.dot(N.x), v_0.dot(N.y), t), dict=True)
Asol, vxsol, vysol, tsol = System_solution[0][A], System_solution[0][v_0.dot(N.x)], System_solution[0][v_0.dot(N.y)], System_solution[0][t]

In [23]:
display(sympy.Eq(A, simp(Asol)))
display(sympy.Eq(v_0.dot(N.x), simp(vxsol)))
display(sympy.Eq(v_0.dot(N.y), simp(vysol)))
display(sympy.Eq(t, simp(tsol)))
display(sympy.Eq(sigma_0,linearization))

         ⎛            2                 2⎞              ⎛            2        
         ⎜ m⋅{\Delta}v       m⋅{\Delta}v ⎟              ⎜ m⋅{\Delta}v       m⋅
         ⎜ ────────────      ────────────⎟              ⎜ ────────────      ──
         ⎜    2⋅k⋅t₀             k⋅t₀    ⎟            2 ⎜    2⋅k⋅t₀           
    - M₂⋅⎝ℯ             + 2⋅ℯ            ⎠ + {\Delta}v ⋅⎝ℯ             + 4⋅ℯ  
A = ──────────────────────────────────────────────────────────────────────────
                          ⎛                2               2                 2
                          ⎜   3⋅m⋅{\Delta}v     m⋅{\Delta}v       m⋅{\Delta}v 
                          ⎜   ──────────────    ────────────      ────────────
                        2 ⎜       2⋅k⋅t₀           2⋅k⋅t₀             k⋅t₀    
               {\Delta}v ⋅⎝4⋅ℯ               + ℯ             + 4⋅ℯ            

         2⎞
{\Delta}v ⎟
──────────⎟
  k⋅t₀    ⎟
          ⎠
───────────
⎞          
⎟          
⎟          
⎟          
⎠         

                                                         ⎛                2   
                                                         ⎜   3⋅m⋅{\Delta}v    
                                                         ⎜   ──────────────   
                                                         ⎜       2⋅k⋅t₀       
                                            M_{1_x}⋅k⋅t₀⋅⎝4⋅ℯ               + 
v_{x_0} = ────────────────────────────────────────────────────────────────────
              ⎛   ⎛                2                   2               2⎞     
              ⎜   ⎜   3⋅m⋅{\Delta}v       2⋅m⋅{\Delta}v     m⋅{\Delta}v ⎟     
              ⎜   ⎜   ──────────────      ──────────────    ────────────⎟     
              ⎜   ⎜       2⋅k⋅t₀               k⋅t₀             k⋅t₀    ⎟     
          2⋅m⋅⎝M₂⋅⎝4⋅ℯ               + 4⋅ℯ               + ℯ            ⎠ - {\

            2                 2⎞                                 
 m⋅{\Delta}v       m⋅{\Delta}v ⎟                                

                                                         ⎛                2   
                                                         ⎜   3⋅m⋅{\Delta}v    
                                                         ⎜   ──────────────   
                                                         ⎜       2⋅k⋅t₀       
                                            M_{1_y}⋅k⋅t₀⋅⎝4⋅ℯ               + 
v_{y_0} = ────────────────────────────────────────────────────────────────────
              ⎛   ⎛                2                   2               2⎞     
              ⎜   ⎜   3⋅m⋅{\Delta}v       2⋅m⋅{\Delta}v     m⋅{\Delta}v ⎟     
              ⎜   ⎜   ──────────────      ──────────────    ────────────⎟     
              ⎜   ⎜       2⋅k⋅t₀               k⋅t₀             k⋅t₀    ⎟     
          2⋅m⋅⎝M₂⋅⎝4⋅ℯ               + 4⋅ℯ               + ℯ            ⎠ - {\

            2                 2⎞                                 
 m⋅{\Delta}v       m⋅{\Delta}v ⎟                                

           ⎛   ⎛                 2                     2                    2 
           ⎜   ⎜    9⋅m⋅{\Delta}v         7⋅m⋅{\Delta}v        5⋅m⋅{\Delta}v  
           ⎜   ⎜    ──────────────        ──────────────       ────────────── 
         2 ⎜   ⎜        2⋅k⋅t₀                2⋅k⋅t₀               2⋅k⋅t₀     
    -k⋅t₀ ⋅⎝M₂⋅⎝64⋅ℯ               + 240⋅ℯ               + 60⋅ℯ               
t = ──────────────────────────────────────────────────────────────────────────
                            ⎛   ⎛                 2                    2      
                            ⎜   ⎜    9⋅m⋅{\Delta}v        7⋅m⋅{\Delta}v       
                            ⎜   ⎜    ──────────────       ──────────────      
                          2 ⎜   ⎜        2⋅k⋅t₀               2⋅k⋅t₀          
             2⋅m⋅{\Delta}v ⋅⎝M₂⋅⎝32⋅ℯ               + 80⋅ℯ               + 10⋅

                2                     2                     2                 
   3⋅m⋅{\Delta}v         4⋅m⋅{\Delta}v         3⋅m⋅

                                                                     ⎛  2     
                                                                   m⋅⎝vₓ  + v_
            ⎛        ⎛  2      2⎞                               ⎞  ───────────
            ⎜    m⋅t⋅⎝vₓ  + v_y ⎠   m⋅vₓ⋅v_{x_0}   m⋅v_y⋅v_{y_0}⎟      2⋅k⋅t₀ 
\sigma₀ = A⋅⎜1 + ──────────────── - ──────────── - ─────────────⎟⋅ℯ           
            ⎜              2            k⋅t₀            k⋅t₀    ⎟             
            ⎝        2⋅k⋅t₀                                     ⎠             

 2⎞
y ⎠
───
   
   
   
   

## Solution to linearization of gausian distribution in 2D ##

Simplifying the solution using the precomputed factors, $e_0 = e^{\frac{m{\Delta v}^2}{2kt_0}}$, $a_0 =-\frac{e_0 + 2{e_0}^2}{{\Delta v}^2\left(e_0 + 4{e_0}^3 + 4{e_0}^2\right)}$, $a_1 = \frac{\left(e_0 + 4{e_0}^2\right)}{\left(e_0 + 4{e_0}^3 + 4{e_0}^2\right)}$

$b_0 = \frac{kt_0\left(e_0 + 4{e_0}^2 + 4{e_0}^3\right)}{2m}$, $b_1 = {e_0}^2 + 4{e_0}^3 + 4{e_0}^4$, $b_2 = -{\Delta v}^2\left({e_0}^2 + 6{e_0}^3 + 8{e_0}^4\right)$

$c_0 = -\frac{k{t_0}^2}{2m{\Delta v}^2}$, $c_1 = {e_0}^3 + 12{e_0}^4 + 60{e_0}^5 + 160{e_0}^6 + 240{e_0}^7 + 192{e_0}^8 + 64{e_0}^9$, $c_2 = -4{\Delta v}^2\left({e_0}^4 + 10{e_0}^5 + 40{e_0}^6 + 80{e_0}^7 + 80{e_0}^8 + 32{e_0}^9\right)$, $c_3 = \left({e_0}^4 + 10{e_0}^5 + 40{e_0}^6 + 80{e_0}^7 + 80{e_0}^8 + 32{e_0}^9\right)$, $c_4 = -{\Delta v}^2\left({e_0}^4 + 12{e_0}^5 + 56{e_0}^6 + 128{e_0}^7 + 144{e_0}^8 + 64{e_0}^9\right)$

Using the precomputed values the equations for the distribution parameters simplify to,

$$A = a_0 M_2 + a_1$$

$$v_{x_0} = \frac{b_0 M_{1_x}}{M_2 b_1 + b_2}$$

$$v_{y_0} = \frac{b_0 M_{1_y}}{M_2 b_1 + b_2}$$

$$t = c_0 \frac{c_1 M_2 + c_2}{c_3 M_2 + c_4}$$

$$\sigma_0 = A\left(1 - \frac{m\vec{v}\cdot\vec{v}_0}{kt_0} + \frac{m \vec{v}\cdot\vec{v}}{2kt_0}\frac{t}{t_0}\right)e^{\frac{m\vec{v}\cdot\vec{v}}{2kt_0}}$$